In [54]:
import pandas as pd
from bs4 import BeautifulSoup  as bs
from bs4 import MarkupResemblesLocatorWarning
import re
from datetime import datetime
from langdetect import detect
import swifter
import warnings

In [55]:
def is_english(review):
    try:
        if(detect(review) == 'en'):
            return review
    except:
        return None 
    
def clean_string(text,second_run):
    if(type(text) == str):
        text = text.lower()
        text = bs(text,'html.parser').get_text()
        tags = re.compile('<.*?>')
        text = re.sub(tags, '', text)
        rx = re.compile(r"[^A-Za-z0-9\s`]")
        text = rx.sub(' ', text)
        rx = re.compile(" +")
        text = rx.sub(' ', text).strip()
        if(not second_run):
            if len(text.split(" ")) < 10:
                return None
        if(second_run):
            if text == 't':
                return 1
            if text == 'f':
                return 0
            try:
                return (datetime.now() - datetime.strptime(text,"%Y %m %d")).days
            except:
                pass
            try:
                temp = text.split(' ')[0]
                pattern = r'^\d+$'
        
                if re.fullmatch(pattern,temp):
                    return float(temp)
            except ValueError:
                pass
        if text == '':
            return None
    return text

In [60]:
df_listings = pd.read_csv('./listings.csv')
df_reviews = pd.read_csv('./reviews.csv')
df_listings = df_listings.convert_dtypes()
df_reviews = df_reviews.convert_dtypes()
df_listings = df_listings.rename(columns={'id':'listing_id'})


In [61]:
#to drop
cols_to_drop_listings = ['listing_url','scrape_id','last_scraped','source','picture_url','host_url','host_name','host_thumbnail_url',
                'host_picture_url','has_availability','license', 'availability_30','availability_60','availability_90','availability_365',
                'calendar_last_scraped','host_location','bathrooms_text','calendar_updated','host_id','host_neighbourhood']

cols_to_drop_reviews = ['reviewer_name','reviewer_id']

In [62]:
warnings.filterwarnings("ignore")
df_listings.drop(columns=cols_to_drop_listings,inplace=True)
df_reviews.drop(columns=cols_to_drop_reviews,inplace=True)
df_reviews['comments'] = df_reviews['comments'].swifter.apply(lambda x: clean_string(x, False))
df_reviews.dropna(subset=['comments'],inplace=True)
df_reviews['comments'] = df_reviews['comments'].swifter.apply(is_english)
df_reviews.dropna(subset=['comments'],inplace=True)
print(len(df_reviews))
df_reviews_combined = df_reviews.groupby('listing_id')['comments'].agg('``'.join).reset_index()
row_counts = df_reviews.groupby('listing_id').size().reset_index(name='review_count')
df_reviews = pd.merge(df_reviews_combined, row_counts, on='listing_id').reset_index()
df_combined = pd.merge(df_reviews,df_listings,on='listing_id').reset_index()
df_combined.drop(columns=['level_0','index'],inplace=True)
df_combined['host_response_rate'] = df_combined['host_response_rate'].str.rstrip('%').astype('float') / 100

Dask Apply:   0%|          | 0/25 [00:00<?, ?it/s]

/home/user/.pyenv/versions/3.12.4/lib/python3.12/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/home/user/.pyenv/versions/3.12.4/lib/python3.12/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/tmp/ipykernel_43820/1954784894.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
/tmp/ipykernel_43820/1954784894.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle

Pandas Apply:   0%|          | 0/218847 [00:00<?, ?it/s]

181717


In [63]:
df_combined = df_combined.applymap(lambda x: clean_string(x,True)).convert_dtypes()

Pandas Apply:   0%|          | 0/252728 [00:00<?, ?it/s]

In [64]:
df_combined.to_csv('combined_short_removed_en_only.csv',index=False,encoding='utf-8')